# Flux Footprint Climatology Tutorial
This Jupyter notebook demonstrates how to use the custom `FFPclim` class (defined in `new_ffp.py`) to compute a flux footprint climatology from tower‐based micrometeorological data.

You will learn how to:
1. Load and inspect the half‐hourly data provided in the CSV file.
2. Instantiate the `FFPclim` model with sensible parameters.
3. Run the model to generate a 2‑D footprint climatology.
4. Visualise and save the results for further analysis.

The workflow should run on a standard scientific Python stack with `pandas`, `numpy`, `scipy`, `xarray`, and `matplotlib` installed.

## 1. Environment setup
Make sure you have the necessary libraries. If something is missing, install it (for example with `pip`).

In [ ]:
!pip install -q pandas numpy scipy xarray matplotlib

## 2. Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
from new_ffp import FFPclim

plt.rcParams['figure.figsize'] = (6, 5)

## 3. Load the half‑hourly data

In [ ]:
DATA_PATH = Path('/mnt/data/US-UTE_HH_202406241430_202409251400.csv')
df_raw = pd.read_csv(DATA_PATH, parse_dates=['TIMESTAMP'], index_col='TIMESTAMP')
print(f"Loaded {len(df_raw):,} records")
df_raw.head()

### 3.1 Clean and inspect key columns
The `FFPclim` class expects certain column names. Below we rename the most common AmeriFlux/ICOS style names if necessary.

In [ ]:
# Example mapping – adjust if your column names differ
col_map = {
    'WD': 'wind_dir',          # wind direction (deg)
    'USTAR': 'USTAR',          # friction velocity (m s‑1)
    'WS': 'ws',                # mean wind speed (m s‑1)
    'SIGMA_V': 'V_SIGMA',      # std dev of lateral wind (m s‑1)
    'MO_LENGTH': 'MO_LENGTH',  # Obukhov length (m)
}
df = df_raw.rename(columns={k:v for k,v in col_map.items() if k in df_raw.columns})

# Drop rows with obviously bad or missing data in key columns
key_cols = ['wind_dir', 'USTAR', 'ws', 'V_SIGMA', 'MO_LENGTH']
df = df.dropna(subset=key_cols)
df.head()

## 4. Instantiate `FFPclim`

In [ ]:
ffp = FFPclim(
    df=df,
    domain=[-1000, 1000, -1000, 1000],  # metres from tower
    dx=2, dy=2,
    smooth_data=True,
    verbosity=1,
)
ffp

## 5. Run the footprint calculation

In [ ]:
results = ffp.run()
results.keys()

## 6. Visualise the footprint climatology

In [ ]:
fclim = results['fclim_2d']

plt.figure()
pc = plt.pcolormesh(ffp.x, ffp.y, fclim.T, shading='auto', cmap='viridis')
plt.colorbar(pc, label='Footprint weight')
plt.contour(ffp.x, ffp.y, fclim.T / fclim.sum(), levels=[0.1,0.2,0.3,0.5,0.8], colors='w')
plt.title('Flux footprint climatology')
plt.xlabel('x (m)'); plt.ylabel('y (m)')
plt.tight_layout()

## 7. Save results

In [ ]:
nc_path = Path('footprint_climatology.nc')
fclim.to_netcdf(nc_path)
print(f'Saved NetCDF to {nc_path.resolve()}')

## 8. Next steps
- Compare different months or stability classes.
- Overlay the footprint with land‑cover maps or lidar canopy height models.
- Couple the footprint to flux partitioning or inverse modelling workflows.